In [3]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report , accuracy_score

In [4]:
df=pd.read_csv('brca_cancer.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'brca_cancer.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
dfe=df.loc[:,['gene_name','tpm_unstranded','class']]

In [ ]:
dfe.head()

In [ ]:
len(dfe['gene_name'].unique()) # Total number of unique genes in an organ

In [ ]:
dfe['gene_name'].value_counts()

In [ ]:
dfe[dfe['gene_name']=='PIK3CA']  # PIK3CA Gene

In [ ]:
dfe[dfe['gene_name']=='TP53']

In [ ]:
dfe[dfe['gene_name']=='BRCA1']

In [ ]:
dfe[dfe['gene_name']=='NF1']

In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
# Encode labels in column 'species'
dfe['gene_name']=encoder.fit_transform(dfe['gene_name'])

In [ ]:
class0 = set(dfe[dfe['class'] == 0]['gene_name'])
class1 = set(dfe[dfe['class'] == 1]['gene_name'])

genes = list(class0.intersection(class1))
new_df = dfe[dfe['gene_name'].isin(genes)]
new_df.head()

In [ ]:
#Checking the unique values
uniq  = new_df['gene_name'].nunique()
uniq

In [ ]:
X_ = new_df.drop(['gene_name', 'class'], axis=1)
y = new_df['class']

population_size = 50
num_generations = 8
top_genes_to_select = min(30, len(X_.columns))

def fitness_function(selected_genes):
    X_selected = X_[:, selected_genes]

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

    dtree = DecisionTreeClassifier(random_state=42)
    dtree.fit(X_train, y_train)

    y_pred = dtree.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    return accuracy

def genetic_algorithm():
    population = [random.sample(list(X_.columns), top_genes_to_select) for _ in range(population_size)]

    for generation in range(num_generations):

        fitness_scores = [fitness_function(individual) for individual in population]


        top_individuals_indices = np.argsort(fitness_scores)[-top_genes_to_select:]
        top_individuals = [population[i] for i in top_individuals_indices]

        new_generation = []
        for _ in range(population_size):
            parent1, parent2 = random.choices(top_individuals, k=2)
            crossover_point = random.randint(0, top_genes_to_select - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            mutation_prob = 0.1
            if random.random() < mutation_prob:
                gene_to_mutate = random.randint(0, top_genes_to_select - 1)
                mutated_gene = random.choice(list(X_.columns))
                child[gene_to_mutate] = mutated_gene
            new_generation.append(child)

        population = new_generation

    best_genes = population[np.argmax(fitness_scores)]

    return best_genes

best = genetic_algorithm()

In [ ]:
y= new_df['class']
X= X_[best]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
dtree=DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train,y_train)

In [ ]:
y_predictions=dtree.predict(X_test)

In [ ]:
print(classification_report(y_test,y_predictions))